**HW 5 - LSTM**

Generating Melodies

In [10]:
import torch
from torch import nn

import music21 as m21
import numpy as np
import pandas as pd
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn.functional as F
from sklearn.preprocessing import OneHotEncoder
warnings.filterwarnings("warning")
from fractions import Fraction
from typing import Dict, List, Optional, Sequence, Tuple

Connect Drive

In [11]:
#getting from drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Getting Path

In [12]:
music_path = r"/content/drive/MyDrive/midi"

Setting Music Durations

In [13]:
music_duration=[0.25,
                0.55,
                0.75,
                1.3,
                1.5,
                2.75,
                3.,
                4.]

#ayarlanan music dosyasının uzantısını algılayabilmesi için
# using music file = .midi extension
def load(music_path):
    musics = []
    for process in os.listdir(music_path):
        warnings.filterwarnings("warning")
        if  process[-3:] == "mid":
            musics.append(m21.converter.parse((music_path+"\{}").format(process)))
    return musics

In [14]:
def data(musics):
    x = []
    size= []
    for process in musics:
        for b in process:
            notes = [a for a in b.flat.notesAndRests]
            b = 0
            for i in notes:
                b = b+1
                if isinstance(i,m21.note.Note):
                    x.append([i.pitch.midi,float(i.m_duration.quarterLength)])
                else:
                    x.append([0,float(i.m_duration.quarterLength)])
            size.append(batch)
    dataset = pd.DataFrame(x,columns=["First column", "Second column"])
    return dataset,size

# major ve minor of musical notes 
def transpose(process):
    notas = process.analyze("notas")
    if notas.mode == "minor":
        interval = m21.interval.Interval(notas.tonic,m21.pitch.Pitch("A"))
    if notas.mode == "major":
        interval = m21.interval.Interval(notas.tonic,m21.pitch.Pitch("C"))
    
    return process.transpose(interval)  

In [9]:
def m_duration(process,music_duration):
    for note in  process.flat.notesAndRests:
        if note.m_duration.quarterLength not in music_duration:
            return False
    return True

def preprocess(music_path,music_duration):
    musics = load(music_path)
    b = []
    for process in musics:
        if not m_duration(process,music_duration):
            continue
        b.append(transpose(process))
    
    return b

In [ ]:
def transform(model,data):
        for d in data:
            b = []
            b.append(model.ohe.transform(d).toarray())
        return np.array(b)

# one hot encoding

def batch(encoded,samp_per_batch=12, sequation_lenght = 10):
    note_per_batch = samp_per_batch * sequation_lenght
    num_batches = int(len(encoded)/note_per_batch)
    encoded = encoded[:num_batches * note_per_batch]
    state = True
    for n in range(0, (encoded.shape[0]-sequation_lenght), sequation_lenght):
        if state:
            x = np.array([encoded[n:n+sequation_lenght]])
            y = np.array([encoded[n+1:n+sequation_lenght+1]])
            state = False
            n2 = 0
        else:
            n2 +=1
            x = np.append(x,np.array([encoded[n:n+sequation_lenght]]),axis=0)
            try:
                y= np.append(y,np.array([encoded[n+1:n+sequation_lenght+1]]),axis=0)
            except:
                y= np.append(y,np.array([encoded[n+1:n+sequation_lenght]]),axis=0)
                y= np.append(y,np.array([[encoded[0]]]),axis=0)

In [ ]:
def output(model,arr):
    model.eval()
    arr = transform(model,np.array([arr]))
    hidden = model.hidden_state(1)
    inputs = torch.from_numpy(arr).float()
    inputs = inputs.cuda()
    lstm_output, hidden = model.forward(inputs,hidden)
    lstm_output,_ = model(inputs,hidden)
    q = lstm_output.cpu().detach().clone().numpy()
    return model.ohe.inverse_transform(q)

In [ ]:
process = preprocess(music_path,music_duration)
dataset = data(process)
b = data(process)

In [ ]:
# Setting Neural Network (LSTM)

class Model_(nn.Module):
    
    def __init__(self, dataset, num_hidden, num_layers, drop, use_gpu):
    #setting class
        num_hidden=256
        drop=0.5
        num_layers=4
        super().__init__()
        self.drop_prob = drop
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.use_gpu = use_gpu
        
        #Setting Encoder       
        self.dataset = dataset
        self.ohe = OneHotEncoder(sparse=True)
        self.ohe.fit_transform(dataset)
        self.uni = len(self.ohe.get_feature_names())
             
        #using LSTM (from research)
        self.lstm = nn.LSTM(self.uni, num_hidden, num_layers, dropout=drop,batch_first=True)
        
        self.dropout = nn.Dropout(drop)
        
        self.fc_linear = nn.Linear(num_hidden, self.uni)
      
    
    def forward(self, x, hidden):
                        
        lstm_output, hidden = self.lstm(x, hidden)      
        drop_output = self.dropout(lstm_output)       
        drop_output = drop_output.contiguous().view(-1, self.num_hidden)
        
        
        final_out = self.fc_linear(drop_output)        
        return final_out, hidden
    
    
    def hidden_state(self, batch_size):
        '''
        Used as separate method to account for both GPU and CPU users.
        '''
        
        if self.use_gpu:
            
            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda(),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda())
        else:
            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden))
        
        return hidden
    

In [ ]:
model = Model_(dataset=dataset, num_hidden=50, num_layers=2, drop_prob=0.5, use_gpu=True)

#using Adam
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss()
train_dataset= dataset.values
test_dataset= dataset.values

Training

In [ ]:
epochs_size = 300
sequation_lenght = 20
batch_size = 5
step = 0

# Set model to train
model.train()
if model.use_gpu:
    model = model.cuda()

for i in range(epochs_size):    
    hidden = model.hidden_state(batch_size)
    for x,y in batch(train_dataset,batch_size,sequation_lenght):
        
        step += 1
        
        # One Hot Encode incoming data
        x = transform(model,x)
        y = transform(model,y)
        # Convert Numpy Arrays to Tensor
        
        inputs = torch.from_numpy(x).float()
        targets = torch.from_numpy(y).float()
               
        if model.use_gpu:
            
        # Reset Hidden State
        # If we dont' reset we would backpropagate through all training history
          hidden = tuple([state.data for state in hidden])
        
          model.zero_grad()
        
          lstm_output, hidden = model.forward(inputs,hidden)
          loss = criterion(lstm_output,targets.view(batch_size*sequation_lenght,-1))
        
          loss.backward()
        
        # POSSIBLE EXPLODING GRADIENT PROBLEM!
        # LET"S CLIP JUST IN CASE
          nn.utils.clip_grad_norm_(model.parameters(),max_norm=5)
        
          optimizer.step()
        
 #chechking on test_dataset      
        if step % 25 == 0:
            
            val_hidden = model.hidden_state(batch_size)
            val_losses = []
            model.eval()
            
            for x,y in batch(test_dataset,batch_size,sequation_lenght):
                
                # One Hot Encode incoming data
                x = transform(model,x)
                y = transform(model,y)

                # Convert Numpy Arrays to Tensor

                inputs = torch.from_numpy(x).float()
                targets = torch.from_numpy(y).float()

                # Adjust for GPU if necessary

                if model.use_gpu:

                    inputs = inputs.cuda()
                    targets = targets.cuda()
                    
                # Reset Hidden State
                # If we dont' reset we would backpropagate through 
                # all training history
                val_hidden = tuple([state.data for state in val_hidden])
                
                lstm_output, val_hidden = model.forward(inputs,val_hidden)
                val_loss = criterion(lstm_output,targets.view(batch_size*sequation_lenght,-1))
        
                val_losses.append(val_loss.item())
            
            # Reset to training model after val for loop
            model.train()
            
            print(f"Epoch: {i} Step: {step} Val Loss: {val_loss.item()}")

Epoch: 0 Step: 25 Val Loss: 4.918031692504883
Epoch: 0 Step: 50 Val Loss: 3.9465954303741455
Epoch: 0 Step: 75 Val Loss: 3.764733076095581
Epoch: 0 Step: 100 Val Loss: 3.708934783935547
Epoch: 1 Step: 125 Val Loss: 3.668837785720825
Epoch: 1 Step: 150 Val Loss: 3.7347631454467773
Epoch: 1 Step: 175 Val Loss: 3.702684164047241
Epoch: 1 Step: 200 Val Loss: 3.764221429824829
Epoch: 2 Step: 225 Val Loss: 3.604902505874634
Epoch: 2 Step: 250 Val Loss: 3.67340087890625
Epoch: 2 Step: 275 Val Loss: 3.59416127204895
Epoch: 2 Step: 300 Val Loss: 3.61981463432312
Epoch: 3 Step: 325 Val Loss: 3.5099918842315674
Epoch: 3 Step: 350 Val Loss: 3.4956114292144775
Epoch: 3 Step: 375 Val Loss: 3.5660207271575928
Epoch: 3 Step: 400 Val Loss: 3.585008382797241
Epoch: 3 Step: 425 Val Loss: 3.5726890563964844
Epoch: 4 Step: 450 Val Loss: 3.430044412612915
Epoch: 4 Step: 475 Val Loss: 3.4965248107910156
Epoch: 4 Step: 500 Val Loss: 3.541926860809326
Epoch: 4 Step: 525 Val Loss: 3.618281126022339
Epoch: 5 Ste

Epoch: 39 Step: 4225 Val Loss: 1.9103710651397705
Epoch: 39 Step: 4250 Val Loss: 1.922908902168274
Epoch: 39 Step: 4275 Val Loss: 2.01640248298645
Epoch: 39 Step: 4300 Val Loss: 1.964058756828308
Epoch: 40 Step: 4325 Val Loss: 1.9069417715072632
Epoch: 40 Step: 4350 Val Loss: 1.8944100141525269
Epoch: 40 Step: 4375 Val Loss: 1.958102822303772
Epoch: 40 Step: 4400 Val Loss: 1.9306107759475708
Epoch: 40 Step: 4425 Val Loss: 1.9208053350448608
Epoch: 41 Step: 4450 Val Loss: 1.86720871925354
Epoch: 41 Step: 4475 Val Loss: 1.8928847312927246
Epoch: 41 Step: 4500 Val Loss: 1.9044039249420166
Epoch: 41 Step: 4525 Val Loss: 1.9232709407806396
Epoch: 42 Step: 4550 Val Loss: 1.8599663972854614
Epoch: 42 Step: 4575 Val Loss: 1.8798044919967651
Epoch: 42 Step: 4600 Val Loss: 1.926985263824463
Epoch: 42 Step: 4625 Val Loss: 1.911041498184204
Epoch: 43 Step: 4650 Val Loss: 1.8501887321472168
Epoch: 43 Step: 4675 Val Loss: 1.84364652633667
Epoch: 43 Step: 4700 Val Loss: 1.9022005796432495
Epoch: 43 S

Epoch: 77 Step: 8350 Val Loss: 1.5800155401229858
Epoch: 77 Step: 8375 Val Loss: 1.5806851387023926
Epoch: 77 Step: 8400 Val Loss: 1.5799442529678345
Epoch: 78 Step: 8425 Val Loss: 1.565329909324646
Epoch: 78 Step: 8450 Val Loss: 1.5648024082183838
Epoch: 78 Step: 8475 Val Loss: 1.5848958492279053
Epoch: 78 Step: 8500 Val Loss: 1.5742727518081665
Epoch: 78 Step: 8525 Val Loss: 1.5690034627914429
Epoch: 79 Step: 8550 Val Loss: 1.557605266571045
Epoch: 79 Step: 8575 Val Loss: 1.56142258644104
Epoch: 79 Step: 8600 Val Loss: 1.5612282752990723
Epoch: 79 Step: 8625 Val Loss: 1.5756860971450806
Epoch: 80 Step: 8650 Val Loss: 1.5561553239822388
Epoch: 80 Step: 8675 Val Loss: 1.560204267501831
Epoch: 80 Step: 8700 Val Loss: 1.5672948360443115
Epoch: 80 Step: 8725 Val Loss: 1.5674248933792114
Epoch: 81 Step: 8750 Val Loss: 1.5596959590911865
Epoch: 81 Step: 8775 Val Loss: 1.5639289617538452
Epoch: 81 Step: 8800 Val Loss: 1.5645986795425415
Epoch: 81 Step: 8825 Val Loss: 1.5629208087921143
Epoch

Epoch: 114 Step: 12400 Val Loss: 1.5078980922698975
Epoch: 115 Step: 12425 Val Loss: 1.4928823709487915
Epoch: 115 Step: 12450 Val Loss: 1.4976779222488403
Epoch: 115 Step: 12475 Val Loss: 1.5026192665100098
Epoch: 115 Step: 12500 Val Loss: 1.5040398836135864
Epoch: 115 Step: 12525 Val Loss: 1.4932411909103394
Epoch: 116 Step: 12550 Val Loss: 1.490884780883789
Epoch: 116 Step: 12575 Val Loss: 1.5036674737930298
Epoch: 116 Step: 12600 Val Loss: 1.49900484085083
Epoch: 116 Step: 12625 Val Loss: 1.5009026527404785
Epoch: 117 Step: 12650 Val Loss: 1.4842610359191895
Epoch: 117 Step: 12675 Val Loss: 1.498829960823059
Epoch: 117 Step: 12700 Val Loss: 1.4911468029022217
Epoch: 117 Step: 12725 Val Loss: 1.5017211437225342
Epoch: 118 Step: 12750 Val Loss: 1.4955227375030518
Epoch: 118 Step: 12775 Val Loss: 1.49647855758667
Epoch: 118 Step: 12800 Val Loss: 1.501038670539856
Epoch: 118 Step: 12825 Val Loss: 1.4992364645004272
Epoch: 118 Step: 12850 Val Loss: 1.4904817342758179
Epoch: 119 Step: 12

Epoch: 151 Step: 16375 Val Loss: 1.4815938472747803
Epoch: 151 Step: 16400 Val Loss: 1.4819433689117432
Epoch: 152 Step: 16425 Val Loss: 1.466693639755249
Epoch: 152 Step: 16450 Val Loss: 1.4728907346725464
Epoch: 152 Step: 16475 Val Loss: 1.473405122756958
Epoch: 152 Step: 16500 Val Loss: 1.4743791818618774
Epoch: 153 Step: 16525 Val Loss: 1.4681147336959839
Epoch: 153 Step: 16550 Val Loss: 1.474959373474121
Epoch: 153 Step: 16575 Val Loss: 1.4814121723175049
Epoch: 153 Step: 16600 Val Loss: 1.4740285873413086
Epoch: 153 Step: 16625 Val Loss: 1.4677218198776245
Epoch: 154 Step: 16650 Val Loss: 1.4612423181533813
Epoch: 154 Step: 16675 Val Loss: 1.4711287021636963
Epoch: 154 Step: 16700 Val Loss: 1.4680534601211548
Epoch: 154 Step: 16725 Val Loss: 1.471130609512329
Epoch: 155 Step: 16750 Val Loss: 1.4615777730941772
Epoch: 155 Step: 16775 Val Loss: 1.4632046222686768
Epoch: 155 Step: 16800 Val Loss: 1.4647600650787354
Epoch: 155 Step: 16825 Val Loss: 1.4744967222213745
Epoch: 156 Step:

Epoch: 188 Step: 20350 Val Loss: 1.4588619470596313
Epoch: 188 Step: 20375 Val Loss: 1.4591995477676392
Epoch: 188 Step: 20400 Val Loss: 1.4571287631988525
Epoch: 189 Step: 20425 Val Loss: 1.4471291303634644
Epoch: 189 Step: 20450 Val Loss: 1.450905680656433
Epoch: 189 Step: 20475 Val Loss: 1.4527751207351685
Epoch: 189 Step: 20500 Val Loss: 1.4578208923339844
Epoch: 190 Step: 20525 Val Loss: 1.4458837509155273
Epoch: 190 Step: 20550 Val Loss: 1.445646047592163
Epoch: 190 Step: 20575 Val Loss: 1.450160026550293
Epoch: 190 Step: 20600 Val Loss: 1.4798758029937744
Epoch: 190 Step: 20625 Val Loss: 1.4517674446105957
Epoch: 191 Step: 20650 Val Loss: 1.457478642463684
Epoch: 191 Step: 20675 Val Loss: 1.4503834247589111
Epoch: 191 Step: 20700 Val Loss: 1.4553642272949219
Epoch: 191 Step: 20725 Val Loss: 1.4530929327011108
Epoch: 192 Step: 20750 Val Loss: 1.4414950609207153
Epoch: 192 Step: 20775 Val Loss: 1.4534295797348022
Epoch: 192 Step: 20800 Val Loss: 1.451406478881836
Epoch: 192 Step: 

Epoch: 225 Step: 24325 Val Loss: 1.4373377561569214
Epoch: 225 Step: 24350 Val Loss: 1.4431836605072021
Epoch: 225 Step: 24375 Val Loss: 1.4360252618789673
Epoch: 225 Step: 24400 Val Loss: 1.4348260164260864
Epoch: 226 Step: 24425 Val Loss: 1.4328052997589111
Epoch: 226 Step: 24450 Val Loss: 1.4400509595870972
Epoch: 226 Step: 24475 Val Loss: 1.4374102354049683
Epoch: 226 Step: 24500 Val Loss: 1.4420114755630493
Epoch: 227 Step: 24525 Val Loss: 1.4337427616119385
Epoch: 227 Step: 24550 Val Loss: 1.437562108039856
Epoch: 227 Step: 24575 Val Loss: 1.4428538084030151
Epoch: 227 Step: 24600 Val Loss: 1.4411636590957642
Epoch: 228 Step: 24625 Val Loss: 1.4340211153030396
Epoch: 228 Step: 24650 Val Loss: 1.4325264692306519
Epoch: 228 Step: 24675 Val Loss: 1.4347875118255615
Epoch: 228 Step: 24700 Val Loss: 1.4377899169921875
Epoch: 228 Step: 24725 Val Loss: 1.4345853328704834
Epoch: 229 Step: 24750 Val Loss: 1.4306416511535645
Epoch: 229 Step: 24775 Val Loss: 1.4357877969741821
Epoch: 229 St

Epoch: 262 Step: 28300 Val Loss: 1.4232409000396729
Epoch: 262 Step: 28325 Val Loss: 1.4217318296432495
Epoch: 262 Step: 28350 Val Loss: 1.4196670055389404
Epoch: 262 Step: 28375 Val Loss: 1.421472430229187
Epoch: 262 Step: 28400 Val Loss: 1.420918345451355
Epoch: 263 Step: 28425 Val Loss: 1.4202375411987305
Epoch: 263 Step: 28450 Val Loss: 1.4308359622955322
Epoch: 263 Step: 28475 Val Loss: 1.4305953979492188
Epoch: 263 Step: 28500 Val Loss: 1.4275624752044678
Epoch: 264 Step: 28525 Val Loss: 1.422402262687683
Epoch: 264 Step: 28550 Val Loss: 1.4248110055923462
Epoch: 264 Step: 28575 Val Loss: 1.4241697788238525
Epoch: 264 Step: 28600 Val Loss: 1.4259803295135498
Epoch: 265 Step: 28625 Val Loss: 1.4213019609451294
Epoch: 265 Step: 28650 Val Loss: 1.4231029748916626
Epoch: 265 Step: 28675 Val Loss: 1.425592064857483
Epoch: 265 Step: 28700 Val Loss: 1.425460934638977
Epoch: 265 Step: 28725 Val Loss: 1.4206184148788452
Epoch: 266 Step: 28750 Val Loss: 1.4211667776107788
Epoch: 266 Step: 

Epoch: 298 Step: 32275 Val Loss: 1.420920968055725
Epoch: 299 Step: 32300 Val Loss: 1.4159923791885376
Epoch: 299 Step: 32325 Val Loss: 1.415917992591858
Epoch: 299 Step: 32350 Val Loss: 1.4138621091842651
Epoch: 299 Step: 32375 Val Loss: 1.4156651496887207
Epoch: 299 Step: 32400 Val Loss: 1.4144048690795898


In [ ]:
torch.save(model.state_dict(),'new_music.net')

In [ ]:
model = Model_(dataset=dataset, num_hidden=50, num_layers=2, drop_prob=0.5, use_gpu=True,)

In [ ]:
noteModel = Model_
a = 'new_music.net'
model.load_state_dict(torch.load(a))
model.eval()

noteModel(
  (lstm): LSTM(29, 50, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_linear): Linear(in_features=50, out_features=29, bias=True)
)

In [ ]:
#her bir değerin karşılığında random bir nota atadığının kanıtı

output(model,dataset[115:125].values)

array([[76.  ,  0.25],
       [74.  ,  0.25],
       [76.  ,  0.25],
       [74.  ,  0.25],
       [74.  ,  0.5 ],
       [74.  ,  0.25],
       [74.  ,  0.25],
       [ 0.  ,  0.5 ],
       [72.  ,  0.25],
       [74.  ,  0.25]])